## Part 1: Preprocessing

In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [4]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [5]:
# Create y_df with the Attrition and Department columns

y_df = attrition_df[['Attrition', 'Department']]

In [9]:
# Create a list of at least 10 column names to use as X data
selected_columns = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany', \
                    'YearsSinceLastPromotion', 'NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[selected_columns]

# Show the data types for X_df
X_df


,Education,Age,DistanceFromHome,JobSatisfaction,OverTime,StockOptionLevel,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,NumCompaniesWorked
0,2,41,1,4,Yes,0,1,6,0,8
1,1,49,8,2,No,1,3,10,1,1
2,2,37,2,3,Yes,0,3,0,0,6
3,4,33,3,3,Yes,0,3,8,3,1
4,1,27,2,2,No,1,3,2,2,9
...,...,...,...,...,...,...,...,...,...,...
1465,2,36,23,4,No,1,3,5,0,4
1466,1,39,6,1,No,1,3,7,1,4
1467,3,27,4,2,Yes,1,3,6,0,1
1468,3,49,2,2,No,0,2,9,0,2


In [11]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df)

In [12]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train['OverTime'].value_counts()

OverTime
No     801
Yes    301
Name: count, dtype: int64

In [13]:
from sklearn.preprocessing import OneHotEncoder

In [17]:
ohe_model = OneHotEncoder(sparse_output=False, drop = 'first', handle_unknown= 'ignore')
ohe_model.fit(np.array(X_train['OverTime']).reshape(-1,1))
X_train['OverTimeEncoded'] = ohe_model.transform(np.array(X_train['OverTime']).reshape(-1,1))
X_test['OverTimeEncoded'] = ohe_model.transform(np.array(X_test['OverTime']).reshape(-1,1))
X_train = X_train.drop('OverTime', axis = 1)
X_test = X_test.drop('OverTime', axis = 1)

In [18]:
# Create a StandardScaler
st_model = StandardScaler()

# Fit the StandardScaler to the training data
st_model.fit(X_train)

# Scale the training and testing data
X_train_scaled = st_model.transform(X_train)
X_test_scaled = st_model.transform(X_test)


In [19]:
# Create a OneHotEncoder for the Department column
ohe_model_2 = OneHotEncoder(sparse_output = False, drop = None, handle_unknown= 'ignore')

# Fit the encoder to the training data
ohe_model_2.fit(np.array(y_train['Department']).reshape(-1,1))

# Create two new variables by applying the encoder
# to the training and testing data
y_department_train = ohe_model.transform(np.array(y_train['Department']).reshape(-1,1))
y_department_test = ohe_model.transform(np.array(y_test['Department']).reshape(-1,1))


C:\Users\nona2\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\nona2\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [20]:
y_department_train

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [21]:
# Create a OneHotEncoder for the Attrition column
ohe_model_attrition = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop = None)

# Fit the encoder to the training data
ohe_model_attrition.fit(np.array(y_train['Attrition']).reshape(-1,1))

# Create two new variables by applying the encoder
# to the training and testing data
y_attrition_train = ohe_model.transform(np.array(y_train['Attrition']).reshape(-1,1))
y_attrition_test = ohe_model.transform(np.array(y_test['Attrition']).reshape(-1,1))


## Create, Compile, and Train the Model

In [23]:
# Find the number of columns in the X training data
number_inputs_features = len(X_train.columns)

# Create the input layer
input = layers.Input(shape = (number_inputs_features,), name = 'input')

# Create at least two shared layers
shared_1 = layers.Dense(64, activation = 'relu', name = 'shared_1')(input)
shared_2 = layers.Dense(128, activation = 'relu', name = 'shared_2')(shared_1)

In [24]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(32, activation = 'relu', name = 'department_hidden')(shared_2)

# Create the output layer
output_layer = layers.Dense(y_department_train.shape[1], activation = 'sigmoid', name = 'output_layer')(department_hidden)


In [25]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32, activation = 'relu', name = 'attrition_hidden')(shared_2)

# Create the output layer
attrition_output = layers.Dense(y_department_train.shape[1], activation = 'sigmoid', name = 'attrition_output')(attrition_hidden)


In [27]:
# Create the model
model = Model(inputs = input, outputs = [output_layer, attrition_output])

# Compile the model
model.compile(optimizer='adam', loss = {'output_layer': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},\
              metrics = {'output_layer': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)            │ (None, 10)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ shared_1 (Dense)              │ (None, 64)                │             704 │ input[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ shared_2 (Dense)              │ (None, 128)               │           8,320 │ shared_1[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ department_hidden (Dense)     │ (None, 32)                │           4,128 │ shared_2[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ attrition_hidden (Dense)      │ (None, 32)                │           4,128 │ shared_2[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ output_layer (Dense)          │ (None, 1)                 │              33 │ department_hidden[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ attrition_output (Dense)      │ (None, 1)                 │              33 │ attrition_hidden[0][0]     │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 17,346 (67.76 KB)

 Trainable params: 17,346 (67.76 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
# Train the model
fit_model = model.fit(X_train_scaled, {'output_layer': y_department_train, 'attrition_output': y_attrition_train}, epochs = 100) 

Epoch 1/100


C:\Users\nona2\anaconda3\Lib\site-packages\keras\src\losses\losses.py:27: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - attrition_output_accuracy: 0.6287 - loss: 0.6407 - output_layer_accuracy: 0.8429
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8185 - loss: 0.5557 - output_layer_accuracy: 1.0000
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8148 - loss: 0.4995 - output_layer_accuracy: 1.0000
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8322 - loss: 0.4533 - output_layer_accuracy: 1.0000
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.7895 - loss: 0.4996 - output_layer_accuracy: 1.0000
Epoch 6/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8145 - loss: 0.5003 - output_layer_accuracy: 1.0000
Epoch 7/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8071 - loss: 0.5174 - output_layer_accuracy: 1.0000
Epoch 8/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accu

In [33]:
# Evaluate the model with the testing data
test_result = model.evaluate(X_test_scaled, {'output_layer': y_department_test, 'attrition_output': y_attrition_test})

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8553 - loss: 0.4849 - output_layer_accuracy: 1.0000 


In [38]:
# Print the accuracy for both department and attrition
print (test_result[:3])
print (test_result[:4])

[0.48567473888397217, 0.85326087474823, 1.0]
[0.48567473888397217, 0.85326087474823, 1.0]


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Data is imbalanced so we can use the F1 score precision or balanced accuracy score.
2. We used the sigmoid funcion and it works for both the attrition and department, tho you can use softmax for department which may be a better choice.
3. By including more X columns or more layers depending on the complexity of the problem.  